In [ ]:
###############################################
# 서울시 소상공인 데이터(공공데이터) 분석 사례
###############################################

In [ ]:
#----------------------
# 1_기본 문법 이해하기
#----------------------

In [ ]:
# 01_외부 파일 읽어오기 (csv,txt, xlsx)

# pd.read_csv(): 판다스 라이브러리의 read_csv() 함수를 사용
# 외부 text 파일, csv 파일을 불러와서 데이터프레임으로 사용할 수 있음

In [ ]:
import pandas as pd

shop_2022 = pd.read_csv('./data/seoul_coffee.csv', encoding='utf-8', sep=',')

In [ ]:
# 02_데이터 이해하기

In [ ]:
# 데이터프레임.head(): 데이터프레임 맨 앞의 부분만 미리보기

shop_2022.head(2)

In [ ]:
# 데이터프레임.shape: 데이터프레임의 행과 열의 갯수를 알려줌

shop_2022.shape

In [ ]:
# print(type(데이터프레임)) : 자료형 확인하기

print(type(shop_2022))

In [ ]:
# 데이터프레임.dtypes: 데이터프레임 내  변수의 형식(타입)을 알려줌

shop_2022.dtypes

In [ ]:
# 데이터프레임.str.contains("문자"): 데이터프레임 내 특정 문자가 포함된 행만 반환
# TRUE / FALSE로 반환

shop_2022['지점명'].str.contains("대학로", na=False)

In [ ]:
# 데이터프레임.str.contains("문자"): 데이터프레임 내 특정 문자가 포함된 행만 반환
# 데이터프레임으로 반환

shop_2022[shop_2022['지점명'].str.contains('대학로', na=False)]

In [ ]:
# 데이터프레임.columns: 데이터프레임의 열 이름을 알려줌

shop_2022.columns

In [ ]:
# (이름으로) 행조회

shop_2022.loc[[1921]]

In [ ]:
# (이름으로) 여러행 동시조회

shop_2022.loc[[1921, 2540]]

In [ ]:
# (이름으로) 열조회

shop_2022.loc[:,'상호명']

In [ ]:
# (이름으로) 열조회

shop_2022.loc[:,'상호명':'상권업종중분류명']

In [ ]:
# (이름으로) 행과열 동시조회

shop_2022.loc[4,'상호명']

In [ ]:
# (위치로) 행조회

shop_2022.iloc[0]

In [ ]:
# (위치로) 열조회

shop_2022.iloc[:,0]

In [ ]:
#-------------------------------------
# 2_판다스 데이터 프레임으로 지도 그리기
#-------------------------------------

In [ ]:
# 01_라이브러리 불러오기

import pandas as pd    # 판다스 데이터프레임 사용 라이브러리
import numpy as np     # 넘파이 사용 라이브러리
import seaborn as sns  # 데이터 시각화 라이브러리
import folium          # 데이터 시각화 라이브러리 

In [ ]:
# 02_소상공인 진흥공단 상가 데이터(서울시) 불러오기

shop_2022 = pd.read_csv('./data/seoul_coffee.csv', encoding='utf-8', sep=',')
shop_2022.head(2)  # 미리보기

In [ ]:
# 03_행렬 수 확인

shop_2022.shape

In [ ]:
# 04_컬럼이름 확인

shop_2022.columns

In [ ]:
# 05_필터링: 특정 지역에 커피 전문점 갯수 알아보기

shop_2022[shop_2022['시군구명'].str.contains('종로', na=False)].shape

In [ ]:
# 06_상호명으로 봤을 때 다양한 유형의 커피 전문점이 있음

shop_2022['상호명'].unique()

In [ ]:
# 07_서울시의 커피 전문점은 모두 13,213개

shop_2022['상호명'].unique().shape

In [ ]:
# 07_커피전문점에서 스타벅스 이름이 포함된 상호만 추출: 466개

sb = shop_2022.loc[shop_2022['상호명'].str.contains('스타벅스')]
sb.shape

In [ ]:
# 08_스타벅스 상호명은...?

shop_2022.loc[shop_2022['상호명'].str.contains('스타벅스'), '상호명'].unique()

In [ ]:
# 09_다른지역의 데이터가 들어가 있는지 확인

sb['시도명'].value_counts()

In [ ]:
# 10_어느 자치구의 데이터들이 있는지 확인

sb['시군구명'].unique()

In [ ]:
# 11_전체 데이터가 몇 개인지 확인

sb['상권업종중분류명'].value_counts()

In [ ]:
# 12_위경도 기준으로 지도에 표현하기

sb.plot.scatter(x="경도", y="위도", grid="True")

In [ ]:
# 13_시군구 기준으로 어느 지역에 가장 많이 있는지 확인

import matplotlib.pyplot as plt
plt.rcParams['font.family'] = 'Malgun Gothic' # 한글폰트(윈도우) 설정
# plt.rcParams['font.family'] = 'AppleGothic'   # 한글폰트(맥) 설정

sns.countplot(data=sb, y="시군구명")

In [ ]:
# 07_마커 클러스터 만들기

import folium
from folium.plugins import MarkerCluster

# 중심점 좌표 찍기
m = folium.Map(location=[37.5642135, 127.0016985], zoom_start=10)

# 마커 위치 리스트업
locations = list(zip(sb.위도, sb.경도))
icons = [folium.Icon(icon="car", prefix="fa") for _ in range(len(locations))]

# 클러스터링 하기
cluster = MarkerCluster(locations=locations, icons=icons)
m.add_child(cluster)
m

In [ ]:
from folium.plugins import HeatMap
map_obj = folium.Map(location=[37.58, 126.97], zoom_start=11) #tiles은 TMS URL, attr은 배경지도 정보

heatMap = HeatMap(zip(sb.위도, sb.경도),                   
                  min_opacity=0.1,
                  radius=20,  #영향 반경 
                  blur=25,     #스무딩 
                  max_zoom=4,
                  color='Red',
                  #gradient={.5: 'Yellow', .75: 'Orange', 1: 'red'}
)
map_obj.add_child(heatMap)
map_obj